Se tienen los Output de dos programas (Kraken y Kaiju), los cuales arrojan , c- Clasificado,  nombre del Read, y el Taxon; 

Objetivo: Hacer un scrpt que tome las salidas de varios programas  de asignacion taxonomica y los ponga en una tabla en la que : (Nombre de Read/ Taxon Asignado por Kraken/  Taxon Asignado por Kaiju) y así para lograr generalizar para varios programas.

1. Poner los datos de los outputs ordenados en la misma tabla...(Nombre/Kraken/Kaiju) 
    1. Leer un (.csv o .tsc) y convertirlo en diccionario.
    1. Hacer archivos mas pequeños de los datos
    1. Crear diccionarios para los datos
    1. Hacer un for para ordenar los datos



In [1]:
#lectura de los datos por medio de pandas, se tomaron las 10 primeras lineas de ambos outputs
import pandas as pd
#datakraken = pd.read_csv('data/EG1BMAA_kraken',sep='\t',header=None, names=["c-clasificado", "read","taxon","1","2"]) 
# pandas.leer archivo .csv(nombre del archivo, separado por tabulaciones, columnas sin nombre)
datakraken10 = pd.read_csv('data/10lineas_kraken',sep='\t',header=None, names=["c-clasificado", "read","taxon","1","2"])
#datakaiju = pd.read_csv('data/EG1BMAA_kaiju',sep='\t',header=None, names=["c-clasificado", "read","taxon"])
datakaiju10 = pd.read_csv('data/10lineas_kaiju',sep='\t',header=None, names=["c-clasificado", "read","taxon"])

In [ ]:
datakraken10

In [ ]:
# como kraken10 tiene dos columnas extra que no necesitamos por el momento las eliminamos con .drop,
# asi quedan los dos outputs con las misma comunas
datakraken101 = datakraken10.drop(['c-clasificado','1', '2'], axis=1)
datakraken101

In [ ]:
datakaiju101 = datakaiju10.drop(['c-clasificado'], axis=1)
datakaiju101

In [ ]:
#Une los DataFrame, pero solo toma el total de los elementos del primero, por lo tanto se pierde informacion.
print (pd.merge(datakaiju101, datakraken101.rename(columns={'read':'read'}), on='read',  how='left'))

In [ ]:
print (pd.merge(datakraken101, datakaiju101.rename(columns={'read':'read'}), on='read',  how='left'))

In [ ]:
datakraken101.info()

In [ ]:
datakaiju101.info()

Cree dos diccionarios, cuyas 'keys' son los reads y los 'values' son los taxones segun la clacificación.

In [ ]:
dickraken = datakraken101.set_index('read').T.to_dict('list')
dickraken

In [ ]:
dickraken['NB501110:123:H7C3WBGX7:1:11101:22148:1069']

In [ ]:
dickaiju = datakaiju101.set_index('read').T.to_dict('list')
dickaiju

In [ ]:
dickaiju['NB501110:123:H7C3WBGX7:1:11101:22148:1069']

In [ ]:
#merge_dicts(dicKraken, dicKaiju)

In [ ]:
#Este for crea un diccionario, tomando todos los datos de ambos diccionarios, mi mostrando el taxon de read para ambos outputs.
from itertools import chain
from collections import defaultdict
dic = defaultdict(list)
for k, v in chain(dickraken.items(), dickaiju.items()):
    dic[k].append(v)
dic 

In [ ]:
dic['NB501110:123:H7C3WBGX7:1:11101:22148:1069']

In [ ]:
#Convertir el diccionario en un dataframe
data_items = dic.items()
data_list = list(data_items)

dfdic = pd.DataFrame(data_list)#, columns=['nombre_read', 'kaiju_taxon'])
dfdic

In [ ]:
#df = pd.DataFrame(dic,columns=['nombre_read','kraken_taxon', 'kaiju_taxon'])
#df

In [ ]:
#dfk = pd.DataFrame(dickaiju, index=['a','b','c'], columns=['nombre_read', 'kaiju_taxon'])
#dfk

In [ ]:
dfk = pd.DataFrame(dickaiju)
dfk
#Esto toma los reads como columnas 

In [ ]:
data_items = dickaiju.items()
data_list = list(data_items)

df = pd.DataFrame(data_list, columns=['nombre_read', 'kaiju_taxon'])
df

In [ ]:
data_items = dickraken.items()
data_list = list(data_items)

dfk = pd.DataFrame(data_list, columns=['nombre_read', 'kraken_taxon'])
dfk

In [ ]:
#Une los DataFrame, pero solo toma el total de los elementos del primero, por lo tanto se pierde informacion.
print (pd.merge(df, dfk, left_on='nombre_read', right_on='nombre_read', how='left'))

In [ ]:
#Une dos diccionarios en uno solo, tomando todos los datos de ambos diccionarios
def mergeDict(dict1, dict2):
   #''' Merge dictionaries and keep values of common keys in list'''
   dic = {**dickraken, **dickaiju}
   for key, value in dic.items():
       if key in dickraken and key in dickaiju:
               dic[key] = [value , dickraken[key]]
   return dic
# Merge dictionaries and add values of common keys in a list
dic = mergeDict(dickraken, dickaiju)
dic

In [ ]:
data_items = dic.items()
data_list = list(data_items)
dfdic = pd.DataFrame(data_list)

In [ ]:
keyskraken = dict.keys(dickraken)
keyskaiju = dict.keys(dickaiju)

In [ ]:
keyskraken & keyskaiju

In [ ]:
keyskraken - keyskaiju

In [ ]:
keyskaiju - keyskraken 

In [ ]:
def mergeDict(dict1, dict2):
   #'Merge dictionaries and keep values of common keys in list'
    keys1 = dict.keys(dict1)
    keys2 = dict.keys(dict2)
    interccion = keys1 & keys2
    dif1 = keys1 - keys2
    dif2 = keys2 - keys1
    dicc = {"read":[],"kraken_taxon":[],"kaiju_taxon":[],"measure":[]}
   #dic = {**dict1, **dict2}
    for i in interccion:
        dicc["read"].append(i)
        dicc["kraken_taxon"].append(dict1[i])
        dicc["kaiju_taxon"].append(dict2[i])
        if dict1[i] == dict2[i]:
            dicc["measure"].append(1)
        dicc["measure"].append(0)
    for i in dif1:
        dicc["read"].append(i)
        dicc["kraken_taxon"].append(dict1[i])
        dicc["kaiju_taxon"].append('NA')
        dicc["measure"].append(0)  
    for i in dif2:
        dicc["read"].append(i)
        dicc["kraken_taxon"].append('NA')
        dicc["kaiju_taxon"].append(dict2[i])
        dicc["measure"].append(0)  
    return dicc
# Merge dictionaries and add values of common keys in a list
dicc = mergeDict(dickraken, dickaiju)
dicc

In [ ]:
#convierte el diccionario en data frame
dfdicc = pd.DataFrame(dicc)
dfdicc